In [2]:
# pip install chromadb sentence-transformers
import chromadb
from chromadb.utils import embedding_functions

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-mpnet-base-v2", device="cuda"
)

chroma_client = chromadb.PersistentClient(path="/home/ansel/chromadb")
collection = chroma_client.get_collection(
    name="roderick", embedding_function=sentence_transformer_ef
)

In [3]:
import os
from transcriptTools import datesDict, titlesDict

ROOT = os.getcwd()

rotl_titles = titlesDict("rotl_titles.txt")
rotl_dates = datesDict("rotl_dates.txt")
roadwork_titles = titlesDict("roadwork_titles.txt")
roadwork_dates = datesDict("roadwork_dates.txt")

In [4]:
chunkedDir = os.path.join(ROOT, "chunked")
dirs = [(os.path.join(chunkedDir, dir), dir) for dir in sorted(os.listdir(chunkedDir))]

files = [
    (os.path.join(dir, file), showname, file.split("_-_")[0], file)
    for dir, showname in dirs
    for file in sorted(os.listdir(dir))
]

In [5]:
for file in files:
    filepath, showname, episode, filename = file
    chunks = open(filepath, encoding="utf-8").read().split("\n\n")
    podcast = "Roderick on the Line" if showname == "rotl" else "Roadwork"
    hosts = (
        "John Roderick and Merlin Mann"
        if showname == "rotl"
        else "John Roderick and Dan Benjamin"
    )

    documents = chunks
    metadatas = [
        {
            "podcast": podcast,
            "hosts": hosts,
            "episode": episode,
            "title": rotl_titles[episode],
            "date": rotl_dates[episode],
        }
        for idx in enumerate(chunks)
    ]
    ids = [f"{showname}_{episode}_{idx}" for idx, chunk in enumerate(chunks)]
    collection.add(documents=documents, metadatas=metadatas, ids=ids)